In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
import keras
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from scipy.interpolate import interp1d



In [21]:
###############
############# Note that the input of this model is the downsampled data. I opened the data in matlab and downsampled them using 
#the downsample(train_data_long_0var,1200) and then export them as csv files and the result is the input of this function

def pm2d5_pred_model(train_data, test_data, problem_type):
    
    if problem_type==2:
        df=train_data.dropna()
        test=test_data
        dataset = df.values
        split_fraction = 0.8
        train_split = int(split_fraction * int(df.shape[0]))
        data_mean = df[:train_split].mean(axis=0)
        data_std = df[:train_split].std(axis=0)
        def normalize(data, train_split):
            data_mean = data[:train_split].mean(axis=0)
            data_std = data[:train_split].std(axis=0)
            return (data - data_mean) / data_std
        features_key = [
            "hmd",
            "spd",
            "tmp",
            "pm2d5",
        ]


        date_time_key = "time"
        selected_features = [features_key[i] for i in [0, 1, 2, 3]]
        selected_features1 = [features_key[i] for i in [0, 1, 2]]
        features = df[selected_features]
        features.index = df[date_time_key]
        features = normalize(features.values, train_split)
        features = pd.DataFrame(features)
        norm = test[selected_features1]
        norm.index = test[date_time_key]
        norm = normalize(norm.values, test.shape[0])
        norm = pd.DataFrame(norm)
        train_data = features.loc[0 : train_split - 1]
        test_data = features.loc[train_split:]
        train_split1 = int(0.5* int(test_data.shape[0]))
        test_data1 = features.loc[train_split : train_split+train_split1 - 1]
        val_data = features.loc[train_split1+train_split:]
        tes=norm.loc[:]
        start = 1
        batch_size = 512 
        epochs = 10
        step = 1
        end = start + train_split
        x_train = train_data[[i for i in range(3)]].values
        y_train = features.iloc[start:end][[3]]
        sequence_length = 2
        x_end = len(val_data)+len(test_data1) - start
        label_start = train_split+train_split1 + start-1
        x_val = val_data.iloc[:x_end][[i for i in range(3)]].values
        y_val = features.iloc[label_start:][[3]]
        dataset_val = keras.preprocessing.timeseries_dataset_from_array(
            x_val,
            y_val,
            sequence_length=sequence_length,
            sampling_rate=step,
            batch_size=batch_size,
        )
        for batch in dataset_val.take(100):
            inputs, targets = batch
        dataset_train = keras.preprocessing.timeseries_dataset_from_array(
            x_train,
            y_train,
            sequence_length=sequence_length,
            sampling_rate=step,
            batch_size=batch_size,
        )
        end = start + train_split1
        x_test = tes[[i for i in range(3)]].values
        y_test = features.iloc[start:len(x_test)+1][[3]]
        for batch in dataset_val.take(100):
            inputs, targets = batch
        dataset_test = keras.preprocessing.timeseries_dataset_from_array(
            x_test,
            y_test,
            sequence_length=sequence_length,
            sampling_rate=step,
            batch_size=batch_size,
        )
        model = Sequential()
        model.add(LSTM(80, activation='relu', input_shape=(162, 3)))
        model.add(Dense(10, activation='relu'))
        model.add(Dense(1))
        model.compile(optimizer='SGD', loss='mse')
        model.fit(dataset_train,
                  epochs=150,
                  validation_data=dataset_val,verbose=1, batch_size=128)
        predicted = model.predict(dataset_test)
        pred= pd.DataFrame({'Column1': predicted[:, 0]})
        new_index = pred.index[-1] + 1
        pred = pred.append(pd.DataFrame(index=[new_index], data=pred.tail(1).values, columns=pred.columns))
        return (pred*data_std['pm2d5'])+data_mean['pm2d5']
    else:
        print("this model is for problem type 2-longterm-prediction")
        




In [27]:
train_data_long_10var=pd.read_csv('train_data_long_10var.csv')
test_data_long_10var=pd.read_csv('test_data_long_10var.csv')

In [28]:
prediction_data_long_10var=pm2d5_pred_model(train_data_long_10var, test_data_long_10var, 2)

Epoch 1/150
3/3 [==============================] - 2s 234ms/step - loss: 0.9998 - val_loss: 0.6259
Epoch 2/150
3/3 [==============================] - 0s 102ms/step - loss: 0.9977 - val_loss: 0.6231
Epoch 3/150
3/3 [==============================] - 0s 99ms/step - loss: 0.9956 - val_loss: 0.6206
Epoch 4/150
3/3 [==============================] - 0s 95ms/step - loss: 0.9935 - val_loss: 0.6185
Epoch 5/150
3/3 [==============================] - 0s 92ms/step - loss: 0.9916 - val_loss: 0.6169
Epoch 6/150
3/3 [==============================] - 0s 103ms/step - loss: 0.9901 - val_loss: 0.6157
Epoch 7/150
3/3 [==============================] - 0s 91ms/step - loss: 0.9888 - val_loss: 0.6148
Epoch 8/150
3/3 [==============================] - 0s 88ms/step - loss: 0.9876 - val_loss: 0.6139
Epoch 9/150
3/3 [==============================] - 0s 94ms/step - loss: 0.9865 - val_loss: 0.6130
Epoch 10/150
3/3 [==============================] - 0s 113ms/step - loss: 0.9854 - val_loss: 0.6121
Epoch 11/150
3/